In [12]:

cd .

c:\Users\karan\Desktop\Amazon classification Project end to end\smart-reviews-classifier\notebooks


In [16]:
import os

# Create the directory (from inside 'notebooks/') if it doesn't exist
os.makedirs('../data', exist_ok=True)

# Then save the CSV
df.to_csv('../data/cleaned_test.csv', index=False)


In [17]:
import pandas as pd
import bz2
import os

df = pd.read_csv('../data/cleaned_test.csv')
df.head()


,label,text
0,1,Stuning even for the non-gamer: This sound tra...
1,1,The best soundtrack ever to anything.: I'm rea...
2,1,Amazing!: This soundtrack is my favorite music...
3,1,Excellent Soundtrack: I truly like this soundt...
4,1,"Remember, Pull Your Jaw Off The Floor After He..."


In [18]:
df.shape

(3600000, 2)

In [19]:
df_small = df.sample(n=50000, random_state=42).reset_index(drop=True)
df_small.to_csv('../data/cleaned_test_small.csv', index=False)

In [20]:

df = pd.read_csv('../data/cleaned_test_small.csv')

In [21]:
df.shape

(50000, 2)

In [25]:
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Download if not already done
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()

def clean_text(text):
    # Lowercase
    text = text.lower()
    
    # Remove URLs
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)

    # Remove special characters, punctuation, digits
    text = re.sub(r'\d+', '', text)  # remove digits
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Remove non-alphabetic characters and extra spaces
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()

    # Remove stopwords and apply stemming
    tokens = text.split()
    tokens = [stemmer.stem(word) for word in tokens if word not in stop_words]

    return " ".join(tokens)

df['clean_text'] = df['text'].apply(clean_text)
df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,label,text,clean_text
0,0,Expensive Junk: This product consists of a pie...,expens junk product consist piec thin flexibl ...
1,0,"Toast too dark: Even on the lowest setting, th...",toast dark even lowest set toast dark like als...
2,1,Excellent imagery...dumbed down story: I enjoy...,excel imagerydumb stori enjoy disc video stun ...
3,0,Are we pretending everyone is married?: The au...,pretend everyon marri author pretend parent ne...
4,0,Not worth your time: Might as well just use a ...,worth time might well use knife product hold n...


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_text'])
y = df['label']


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

svm_model = LinearSVC(class_weight='balanced')
svm_model.fit(X_train, y_train)

y_pred = svm_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.8782
              precision    recall  f1-score   support

           0       0.88      0.87      0.88      4992
           1       0.88      0.88      0.88      5008

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [76]:
logistic_model = LogisticRegression(class_weight='balanced', max_iter=1000)
logistic_model.fit(X_train, y_train)

y_pred = logistic_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8809
              precision    recall  f1-score   support

           0       0.89      0.87      0.88      4992
           1       0.88      0.89      0.88      5008

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [77]:
multinb_model = MultinomialNB()
multinb_model.fit(X_train, y_train)

y_pred = multinb_model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.8551
              precision    recall  f1-score   support

           0       0.86      0.85      0.85      4992
           1       0.85      0.86      0.86      5008

    accuracy                           0.86     10000
   macro avg       0.86      0.86      0.86     10000
weighted avg       0.86      0.86      0.86     10000



In [30]:
from sklearn.ensemble import VotingClassifier

ensemble = VotingClassifier(estimators=[
    ('lr', LogisticRegression(class_weight='balanced', max_iter=1000)),
    ('svm', LinearSVC(class_weight='balanced')),
], voting='hard')

ensemble.fit(X_train, y_train)
y_pred_ensemble = ensemble.predict(X_test)

In [31]:
print("Ensemble Accuracy:", accuracy_score(y_test, y_pred_ensemble))
print(classification_report(y_test, y_pred_ensemble))

Ensemble Accuracy: 0.8792
              precision    recall  f1-score   support

           0       0.86      0.90      0.88      4992
           1       0.90      0.85      0.88      5008

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



## Hyperparameter Tuning

In [49]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'C': [0.01, 0.1, 1, 10],
    'penalty': ['l2'],
    'solver': ['liblinear', 'saga'],
    'max_iter': [100, 200]
}

In [66]:
# best_params = grid.best_params_
# best_C = best_params['C']


In [51]:
log_reg = LogisticRegression()

grid_search = GridSearchCV(log_reg, param_grid, cv=3, verbose=2, n_jobs=-1, scoring='accuracy')


In [52]:
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


,estimator,LogisticRegression()
,param_grid,"{'C': [0.01, 0.1, ...], 'max_iter': [100, 200], 'penalty': ['l2'], 'solver': ['liblinear', 'saga']}"
,scoring,'accuracy'
,n_jobs,-1
,refit,True
,cv,3
,verbose,2
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,False
,penalty,'l2'


In [53]:
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

Best Parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l2', 'solver': 'liblinear'}
Best Accuracy: 0.8756250329687071


In [54]:
best_model = grid_search.best_estimator_

In [56]:
best_params = grid_search.best_params_
best_C = best_params['C']

In [37]:
y_pred = best_model.predict(X_test)
print("Test Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Test Accuracy: 0.8813
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      4992
           1       0.88      0.89      0.88      5008

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



### XGBoost Algo


In [42]:
# Ensure X_train is raw text
print(type(X_train))


<class 'scipy.sparse._csr.csr_matrix'>


In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier

model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)


c:\Users\karan\Desktop\Amazon classification Project end to end\smart-reviews-classifier\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [12:45:52] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None
,colsample_bynode,None
,colsample_bytree,None
,device,None
,early_stopping_rounds,None
,enable_categorical,False
,eval_metric,'logloss'


In [44]:
y_pred = model.predict(X_test)

In [45]:
print("XGBoost Accuracy:", accuracy_score(y_test, y_pred))

XGBoost Accuracy: 0.844


## Comparing the Models

In [80]:
from sklearn.metrics import accuracy_score
import pandas as pd


results = []

# Logistic Regression
log_reg = LogisticRegression(C=1, max_iter=1000)
log_reg.fit(X_train, y_train)
log_pred = log_reg.predict(X_test)
results.append(["Logistic Regression", accuracy_score(y_test, log_pred)])

# SVM
svm = LinearSVC(C=1)
svm.fit(X_train, y_train)
svm_pred = svm.predict(X_test)
results.append(["SVM", accuracy_score(y_test, svm_pred)])

# XGBoost
xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
results.append(["XGBoost", accuracy_score(y_test, xgb_pred)])

# Multinomial Naive Bayes
multinb_model = MultinomialNB()
multinb_model.fit(X_train, y_train)
multinb_model_pred = multinb_model.predict(X_test)
results.append(["MultinomialNB", accuracy_score(y_test, multinb_model_pred)])


# Ensemble 
ensemble_preds = (log_pred + svm_pred) >= 1  # majority voting
results.append(["Ensemble (LogReg + SVM)", accuracy_score(y_test, ensemble_preds)])

accuracy_df = pd.DataFrame(results, columns=["Model", "Accuracy"])
print(accuracy_df)


c:\Users\karan\Desktop\Amazon classification Project end to end\smart-reviews-classifier\.venv\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:04:47] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


                     Model  Accuracy
0      Logistic Regression    0.8817
1                      SVM    0.8780
2                  XGBoost    0.8440
3            MultinomialNB    0.8551
4  Ensemble (LogReg + SVM)    0.8803


## Final Model


In [57]:
import joblib

final_model = LogisticRegression(C=best_C, max_iter=1000)
final_model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [58]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = final_model.predict(X_test)
print("Final Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Final Accuracy: 0.8817
              precision    recall  f1-score   support

           0       0.89      0.88      0.88      4992
           1       0.88      0.89      0.88      5008

    accuracy                           0.88     10000
   macro avg       0.88      0.88      0.88     10000
weighted avg       0.88      0.88      0.88     10000



In [60]:
os.makedirs('models', exist_ok=True)

joblib.dump(final_model, 'models/logistic_model.joblib')
joblib.dump(tfidf, 'models/tfidf_vectorizer.joblib')

['models/tfidf_vectorizer.joblib']

### Testing the saved Model

In [69]:
import joblib

vectorizer = TfidfVectorizer(max_features=10000, ngram_range=(1, 2))
X = vectorizer.fit_transform(df['clean_text'])

joblib.dump(vectorizer, 'models/tfidf_vectorizer.joblib')



['models/tfidf_vectorizer.joblib']

In [70]:
model = joblib.load('models/logistic_model.joblib')
vectorizer = joblib.load('models/tfidf_vectorizer.joblib')

In [71]:
review = ["This product is amazing and exceeded my expectations!"]

In [72]:
review_vectorized = vectorizer.transform(review)

# Predict
prediction = model.predict(review_vectorized)
print("Prediction:", prediction)

Prediction: [0]


In [74]:
reviews = [
    "Terrible product, complete waste of money.",
    "Absolutely loved it! Highly recommend.",
    "It was okay, nothing special."
]

for r in reviews:
    pred = model.predict(vectorizer.transform([r]))[0]
    print(f"Review: {r} → Prediction: {pred}")


Review: Terrible product, complete waste of money. → Prediction: 0
Review: Absolutely loved it! Highly recommend. → Prediction: 1
Review: It was okay, nothing special. → Prediction: 0
